In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from cornea.models.trainer_module import CorneaTrainerModule
from cornea.models.hf_hubs import submit_model, download_model
import torchseg
import wandb
import os

In [3]:
api = wandb.Api()
project_name = "Cornea Interface Segmentation"
runs = api.runs(f"liv4d-polytechnique/{project_name}")
root_ckpt = Path("../checkpoints") / project_name


for run in runs:
    ckpt = root_ckpt / run.name
    name = [f for f in os.listdir(ckpt) if "last" not in f][0]
    ckpt_path = ckpt / name
    encoder = run.config["encoder"]
    base = torchseg.create_model(run.config["model"], classes=1, encoder_weights=None)
    encoder = base.name.split("-")[1]
    model = CorneaTrainerModule.load_from_checkpoint(ckpt_path, model=base)
    model = model.model
    submit_model(model, run.config["model"], encoder)

model.safetensors:   0%|          | 0.00/97.9M [00:00<?, ?B/s]